In [ ]:
!pip install -q -U git+https://github.com/mljar/mljar-supervised.git@master

In [ ]:
conda install -c conda-forge python-snappy fastparquet snappy

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
import datatable as dt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import classification_report, accuracy_score, average_precision_score
import fastai
from fastai import *
from fastai.tabular.all import *
from sklearn import metrics
from sklearn import preprocessing
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, LabelEncoder
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import train_test_split, KFold , StratifiedKFold, cross_val_score
import optuna
import plotly
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from supervised.automl import AutoML 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

I just combine different ideas from kagglers. You can see some shadow from the codes below.

In [ ]:
def cef(data_0):
    data = data_0.copy()
    age_map = data[['Age', 'Pclass']].dropna().groupby('Pclass').mean().to_dict()
    data.Age = data.Age.fillna(data.Pclass.map(age_map['Age']))
    data.Cabin = data['Cabin'].fillna('X').map(lambda x: x[0].strip())
    data.Ticket = data['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')
    fare_map = data[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
    data['Fare'] = data['Fare'].fillna(data['Pclass'].map(fare_map['Fare']))
    data['Fare'] = np.log1p(data['Fare'])
    data['Embarked'] = data['Embarked'].fillna('X')
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1    
    data['FirstName'] = data.Name.map(lambda x: str(x).split(',')[0])
    data['Surname'] = data.Name.map(lambda x: str(x).split(',')[1])   
    for col in ['Name', 'FirstName', 'Surname']:
        data['Counter_' + col] = data[col].map(data.groupby(col)['PassengerId'].count().to_dict())
    data.drop(columns = ["PassengerId",'Name','Surname'], inplace = True)
    label_cols = ['FirstName', 'Ticket', 'Sex', 'Cabin', 'Embarked', 'Pclass']
    numerical_cols = ['Age', 'SibSp', 'Parch', 'Fare','FamilySize', 'Counter_Name','Counter_FirstName','Counter_Surname']
    def label_encoder(c):
        le = LabelEncoder()
        return le.fit_transform(c)
    #scaler = StandardScaler()
    #onehot_encoded_df = pd.get_dummies(all_df[onehot_cols])
    label_encoded_df = data[label_cols].apply(label_encoder)
    #numerical_df = pd.DataFrame(scaler.fit_transform(all_df[numerical_cols]), columns=numerical_cols)
    target_df = data['Survived']
    #all_df = pd.concat([numerical_df, label_encoded_df, onehot_encoded_df, target_df], axis=1)
    data = pd.concat([data[numerical_cols], label_encoded_df, target_df], axis=1)
    return data

In [ ]:
TARGET = 'Survived'
test_data = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
train_data = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
# file1 = pd.read_csv("../input/proba/proba th 0.5.csv")
# file2 = pd.read_csv("../input/proba/0.816.csv")
test_id = test_data["PassengerId"]
#file = file1.merge(file2, on ="PassengerId", how = "inner")
#file.columns = 'PassengerId', 'proba', 'Survived'
#pseudo_label_test = (file[(file.proba <= 0.1) | (file.proba >= 1)].reset_index(drop = True))
#pseudo_label_test.drop("proba", axis=1, inplace=True)
pseudo_label_test = pd.read_csv("../input/425425425/file 1 0.81759.csv")
pseudo_merge_test = test_data.merge(pseudo_label_test, on = "PassengerId", how = "inner")
df = pd.concat([train_data, pseudo_merge_test]).reset_index(drop = True)
#df.drop("PassengerId", axis=1, inplace=True)
y = df["Survived"]
#test_data.drop("PassengerId", axis=1, inplace=True)
all_df = pd.concat([df, test_data]).reset_index(drop = True)
all_df = cef(all_df)
train_data, test_data = all_df[:len(df)], all_df[len(df):]
X = train_data.drop("Survived", axis=1)
to_test = test_data.drop("Survived", axis=1)
to_test = to_test.reset_index(drop = True)
X_male = X[X["Sex"] == 1]
y_male = y[X_male.index]
X_female = X[X["Sex"] == 0]
y_female = y[X_female.index]
to_test_male = to_test[to_test["Sex"] == 1]
to_test_female = to_test[to_test["Sex"] == 0]
X_male = X_male.drop("Sex", axis = 1)
X_female = X_female.drop("Sex", axis = 1)
to_test_male = to_test_male.drop("Sex", axis = 1)
to_test_female = to_test_female.drop("Sex", axis = 1)
tmp = 200000 - (X_female.shape[0] + X_male.shape[0])
tmp2 = tmp / 100000
print ("filter out", tmp, "unconfident fake label data")
print ("which holds", tmp2 * 100, "% error")
print ("which means you are supposed to have", (1 - tmp2) * 100, "% accuracy right now")
del train_data
del test_data
del all_df

In [ ]:
model_male = AutoML(mode="Compete", 
                    total_time_limit = 1000
                    # 14400
                    ,algorithms = ["Random Forest", "CatBoost","LightGBM", "Extra Trees"], random_state = 42)
model_male.fit(X_male, y_male)
y_pred_male = model_male.predict_proba(to_test_male)[:,1]
to_test.loc[to_test_male.index, "Survived"] = y_pred_male

model_female = AutoML(mode="Compete", 
                    total_time_limit = 1000
                    # 14400
                    ,algorithms = ["Random Forest", "CatBoost","LightGBM", "Extra Trees"], random_state = 42)
model_female.fit(X_female, y_female)
y_pred_female = model_female.predict_proba(to_test_female)[:,1]
to_test.loc[to_test_female.index, "Survived"] = y_pred_female


In [ ]:
threshold = 0.5
#threshold = pd.Series(to_test.Survived).sort_values(ascending = False).head(34911).values[-1]
print(f"Current threshold is: {threshold}")

submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')
submission['submit_1'] = (to_test.Survived > threshold).astype(int)
submission['submit_2'] = pd.read_csv("../input/425425425/dae.csv")[TARGET]
submission['submit_3'] = pseudo_label_test[TARGET]
submission[[col for col in submission.columns if col.startswith('submit_')]].sum(axis = 1).value_counts()
submission[TARGET] = (submission[[col for col in submission.columns if col.startswith('submit_')]].sum(axis=1) >= 2).astype(int)
submission[TARGET].mean()
submission[['PassengerId', TARGET]].to_csv("TPS425.csv", index = False)